Import modules and use multi Processing

In [10]:
import datetime
import pandas_ta as ta
import pandas as pd
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover
import yfinance as yf
import numpy as np
import multiprocessing


multiprocessing.set_start_method('fork')

RuntimeError: context has already been set

## Set up Pandas dateframe


- note MACD_14_28_9 is an example of he defualt name in pandas_ta
- resample is used to calculate MACD values for the higher timeframe
- For the first X rows in the dateframe MACD not calacuuted so we drop these rows to avoid errors using the df.dropna function 


In [11]:
df = yf.download(tickers='AAPL', start='2019-01-01', end='2022-12-31')

if isinstance(df.columns, pd.MultiIndex):
    df.columns = [' '.join(col).strip() for col in df.columns]

df.columns = [col.split(' ')[0] for col in df.columns]
df['200EMA'] = ta.ema(df['Close'], timeperiod=200)
df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)


daily_MACD = ta.macd(df['Close'], fast=14, slow=28, signal=9) #should optimise these 
weekly_df = df['Close'].resample('W-FRI').last()  # Resample to weekly frequency, using Friday as the end of the week
weekly_MACD = ta.macd(weekly_df, fast=14, slow=28, signal=9)  # Apply MACD on the resampled data


if isinstance(daily_MACD, pd.DataFrame):
    df['dailymacd'] = daily_MACD['MACD_14_28_9']
    df['dailymacdsignal'] = daily_MACD['MACDs_14_28_9']
    df['dailymacdhist'] = daily_MACD['MACDh_14_28_9']

if isinstance(weekly_MACD, pd.DataFrame):
    df['weeklymacd'] = weekly_MACD['MACD_14_28_9']
    df['weeklymacdsignal'] = weekly_MACD['MACDs_14_28_9']
    df['weeklymacdhist'] = weekly_MACD['MACDh_14_28_9']


df = df.dropna(how='all')

[*********************100%***********************]  1 of 1 completed


## Create signals
The sell conditions do not have lab els as they are the opposite of the buy conditions

In [12]:
def apply_total_signal(df):
   signals = np.zeros(len(df))
   
   buy_conditions = (
    (crossover(df['dailymacd'], df['dailymacdsignal'])) &  # Daily MACD crossover
    (df['dailymacd'] < 0) &  # Daily MACD below zero (reversal setup)
    (df['weeklymacdhist'] > df['weeklymacdhist'].shift(1)) &  # Weekly MACD histogram rising
    (df['Close'] > df['200EMA']) &  # Price above 200 EMA
    (df['Close'] > df['Close'].shift(1))
     )  # Positive price slope
   
   sell_conditions = (
    (crossover(df['dailymacdsignal'], df['dailymacd'])) &
    (df['dailymacd'] > 0) &  # MACD is above zero
    (df['weeklymacdhist'] < df['weeklymacdhist'].shift(1)) &
    (df['Close'] < df['200EMA']) &
    (df['Close'] < df['Close'].shift(1))
    )
   
   signals[buy_conditions] = 2
   signals[sell_conditions] = 1

   return pd.Series(signals, index=df.index)

## Create the stratergy class and backtest

- Lamdba functions are used to plot the EMA and MACD on the graph
- we optimise the take profit coefficient
- Could also optimise the MACD fast, slow, and signal however for this code I chose not too

In [13]:
class MyStrat(Strategy):
    #stop loss for both is the 200 ema 
    mysize = 0.1
    TPcoef = 2.0


    def init(self):
       super().init()
       # Pre-calculate signals 
       self.signal1 = self.I(self.calculate_signal)
       self.I(lambda: self.data.df['200EMA'], name='200EMA')

       # Plotting 
       self.I(lambda: self.data.df['dailymacd'], name='Daily MACD')
       self.I(lambda: self.data.df['dailymacdsignal'], name='Daily MACD Signal')
       self.I(lambda: self.data.df['dailymacdhist'], name='Daily MACD Histogram')
        
       self.I(lambda: self.data.df['weeklymacd'], name='Weekly MACD')
       self.I(lambda: self.data.df['weeklymacdsignal'], name='Weekly MACD Signal')
       self.I(lambda: self.data.df['weeklymacdhist'], name='Weekly MACD Histogram')

    def calculate_signal(self):
       return apply_total_signal(self.data.df)
    
    def next(self):
       super().next()
      
       # Only calculate if we don't have any open positions
       if not self.position:
           tpatr = self.TPcoef * self.data.ATR[-1]
           sl1 = self.data.df['200EMA'].iloc[-1]

           if self.signal1[-1] == 2:  # Buy signal
               tp1 = self.data.Close[-1] + tpatr
               self.buy(sl=sl1, tp=tp1, size=self.mysize)
              
           elif self.signal1[-1] == 1:  # Sell signal
               tp1 = self.data.Close[-1] - tpatr
               self.sell(sl=sl1, tp=tp1, size=self.mysize)
           

bt = Backtest(df, MyStrat, cash=10000, margin=1/10, commission=0.001)

# Otimisation 
stats = bt.optimize(       
   TPcoef=[2.5, 3.0, 3.5, 4.0, 4.5],          
   maximize='Return [%]'                      
)


print("\nOptimization Results:")
print(stats)
print("\nOptimal Parameters:")
print(f"TP Coefficient: {stats._strategy.TPcoef}")


bt.plot()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/backtesting.py:1384: FutureWarning: The behavior of Series.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  best_params = heatmap.idxmax()
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')



Optimization Results:
Start                     2019-01-02 00:00...
End                       2022-12-30 00:00...
Duration                   1458 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]              229.103329
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade 

Row(id='2834', ...)